In [1]:
import sys, os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.signal
import importlib
import h5py
sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
import readdata

In [13]:
path = "/Users/ajc/Core/simulations/barrow_icewedges/hydrologic_connection/ProjectionsN/"

Simulations = [['CESM_C10','CESM_C11'], ['CESM_C20', 'CESM_C21'],['CESM_C30','CESM_C31','CESM_C32','CESM_C33'] ]# modified rain model
#Simulations = [['CESM_C10'], ['CESM_C20'],['CESM_C30','CESM_C31','CESM_C32','CESM_C33'] ]# modified rain model, dessicated zone =0.2
#Dec 15, 2020
Simulations = [['CESM_C10B','CESM_C11B','CESM_C12B','CESM_C13B'], ['CESM_C20','CESM_C21','CESM_C22'],['CESM_C30','CESM_C31','CESM_C32','CESM_C33','CESM_C34'] ]# modified rain model, dessicated zone =0.1, longwaves


Sim_name = ['S1','S2','S3']
to_day_mm = 86400 * 1000
to_day_year_mm = 86400 * 365 * 1000
to_day_year = 86400 * 365 
to_mm = 1000.
surface_area = 61.92
molar_den = 55500.

In [3]:
def read_file(file):
    d = np.loadtxt(file,usecols=1,skiprows=6)
    t1 = np.loadtxt(file,usecols=0,skiprows=6)
    t = np.array([x/86400./365. for x in t1])
    l = np.minimum(len(d),len(t))
    return d[:l],t[:l]

def read_set(set,var):
    dat = []
    tim = []
    for s in set:
        file = os.path.join(path,s, var)
        #print (file)
        d = np.loadtxt(file,usecols=1,skiprows=6)
        t1 = np.loadtxt(file,usecols=0,skiprows=6)
        t = np.array([x/86400./365. for x in t1])
        
        L = len(d)%365
        if L >0:
            d = d[:-L]
            t = t[:len(d)]
            
        dat.append(d)
        tim.append(t)
    dat = np.concatenate(dat)
    tim = np.concatenate(tim)
    #print ('Lengths: ',len(dat)%365, len(tim)%365,L)
    
    return dat, tim

In [25]:
%matplotlib qt
def get_Discharge():
    time = 2006 + np.linspace(0,94,94,dtype='int')
    Discharge = dict()
    Recharge = dict()
    for i,sim in enumerate(Simulations):
        d1, t1 = read_set(sim,'surface_flux-day.dat')
        #d11 = [x/(molar_den*surface_area)*to_day_mm for x in d1] #converted to day here
        d11 = [x/(molar_den)*to_day_year for x in d1] #converted to day here
        #d11 = [x*86400 for x in d1] #converted to day here
        
        if (i==2):
            d12 = np.reshape(d11,(-1,365))
            d21 = []
            d22 = []
            for d in d12:
                #print (len(d))
                #break
                dx = np.where(d>=0)
                dy = np.where(d<0)
                sum1 = np.sum(d[dx])/len(d)
                sum2 = np.sum(d[dy])/len(d)
                d21.append(sum1)
                d22.append(sum2)
            Discharge[Sim_name[i]] = d21
            Recharge[Sim_name[i]] = d22
        else:
            d2 = np.reshape(d11,(-1,365)).mean(axis=1)#*365
            #t2 = np.reshape(t1,(-1,365)).mean(axis=1)
        
            Discharge[Sim_name[i]] = d2
        
    return Discharge,Recharge

In [26]:
Qout_annual, Qin_annual = get_Discharge()

308 57
300 65
320 45
331 34
273 92
296 69
287 78
294 71
313 52
337 28
321 44
317 48
327 38
325 40
313 52
342 23
329 36
315 50
311 54
321 44
303 62
273 92
284 81
250 115
305 60
315 50
269 96
291 74
289 76
343 22
310 55
282 83
290 75
297 68
315 50
269 96
276 89
303 62
262 103
271 94
297 68
268 97
304 61
314 51
316 49
284 81
323 42
315 50
277 88
291 74
283 82
277 88
239 126
283 82
269 96
271 94
274 91
241 124
283 82
253 112
296 69
287 78
268 97
283 82
284 81
270 95
245 120
258 107
254 111
244 121
248 117
278 87
240 125
287 78
281 84
299 66
250 115
295 70
307 58
265 100
250 115
226 139
243 122
162 203
278 87
252 113
238 127
276 89
215 150
265 100
223 142
260 105
253 112
236 129
256 109


In [27]:
plt.plot(Qout_annual['S1'],'b')
plt.plot(Qout_annual['S2'],'k')
plt.plot(Qout_annual['S3'],'r')
plt.plot(Qin_annual['S3'],'r.')

In [ ]:
def getSurfaceData(i,var,hdf_file='visdump_surface_data.h5',yr_len=365, average=True):
    Data = []
    print ('getSurfaceData: ',var,average)
    for s in Simulations[i]:
        infile = os.path.join(path,s,hdf_file)
        print (infile)
        dx=h5py.File(infile,'r')
        d = np.array(readdata.GetSurfVarFromVis_All(infile,var)[var])
        if (average):
            d = d.mean(axis=1)
        else:
            d = d.sum(axis=1)
        L = len(d)%yr_len
        if L >0:
            d = d[:-L]
            
        Data.append(d)
    d = dict()
    d[var] = np.concatenate(Data)
    
    return d

In [ ]:
def Evaporation():
    Colors = ['k','b','grey','g','r','m']
    time = 2006 + np.linspace(0,94,94,dtype='int')
    d3_sf = []
    sf_change = 0
    Evap = dict()
    for i,sim in enumerate(Simulations):
        var = 'surface-evaporative_flux'
        d1 = getSurfaceData(i, var,yr_len=73)[var]
        d2 = [x*to_day_mm for x in d1]
        d3_sf = [x*to_day_mm for x in d2] 
        Evap[Sim_name[i]] = np.reshape(d1,(-1,73)).mean(axis=1)*to_day_year_mm
        print (len(Evap[Sim_name[i]]))
    return Evap

In [ ]:
E_annual = Evaporation()

In [ ]:
plt.plot(E_annual['S1'])
plt.plot(E_annual['S2'])
plt.plot(E_annual['S3'])

In [ ]:
def SF_WaterContent():
    SF_WC = dict()
    for i,sim in enumerate(Simulations):
        var = 'surface-water_content'
        d1_sf = getSurfaceData(i, var,yr_len=73,average=False)[var]
        factor = 1/(molar_den *surface_area)*to_mm
        d2_sf = [x*factor for x in d1_sf]
        d3_sf = np.reshape(d2_sf,(-1,73))
        d_c = []
        for d in d3_sf:
            d_c.append(d[-1] - d[0])
        SF_WC[Sim_name[i]] = d_c
        print (len(SF_WC[Sim_name[i]]))
    return SF_WC

In [ ]:
SF_WC = SF_WaterContent()

In [ ]:
plt.plot(SF_WC['S1'])
plt.plot(SF_WC['S2'],'k')
plt.plot(SF_WC['S3'],'r')

In [ ]:

def getSubSurfaceData(i,var='',hdf_file='visdump_data.h5',yr_len=73,average=False):
    Data = []
    print ('getSubSurfaceData: ',var,average)
    for s in Simulations[i]:
        infile=os.path.join(path,s,hdf_file)
        d1_ss = readdata.GetSumSurfVarFromVis(file=infile, var=var)[var]
        L = len(d1_ss)%yr_len
        if L >0:
            d1_ss = d1_ss[:-L]
            
        Data.append(d1_ss)
    d = np.concatenate(Data)
    
    return d

In [ ]:
from multiprocessing import Pool
from functools import partial
num_processors = 3

pool=Pool(processes = num_processors)

def SS_WaterContent():
    SS_WC = dict()
    var = 'water_content'
    prod_x=partial(getSubSurfaceData,var=var,yr_len=73)
    global DataS1, DataS2, DataS3
    DataS1, DataS2, DataS3 = pool.map(prod_x,[i for i in range(len(Simulations))])
    d1 = [DataS1, DataS2,DataS3]
    for i in range(3):
        d1_ss = d1[i]
        factor = 1/(molar_den *surface_area)*to_mm
        d2_ss = [x*factor for x in d1_ss]
        d3_ss = np.reshape(d2_ss,(-1,73))
        d_c = []
        for d in d3_ss:
            d_c.append(d[-1] - d[0])
        SS_WC[Sim_name[i]] = d_c
        print (len(SS_WC[Sim_name[i]]))
    return SS_WC



In [ ]:
SS_WC = SS_WaterContent()

In [ ]:
plt.plot(SS_WC['S1'])
plt.plot(SS_WC['S2'],'k')
plt.plot(SS_WC['S3'],'r')

In [ ]:
def water_content_snow():
    SF_SC = dict()
    for i,sim in enumerate(Simulations):
        var_wc_sf = 'snow-swe'
        d1_sf = getSurfaceData(i, var_wc_sf,hdf_file='visdump_snow_data.h5',yr_len=73,average=False)[var_wc_sf]        
        
        d2_sf = np.reshape(d1_sf,(-1,73))

        d_c = []
        for d in d2_sf:
            x = (d[-1] - d[0])/surface_area*1000
            d_c.append(d[-1] - d[0])
        SF_SC[Sim_name[i]] = d_c
        print (len(SF_SC[Sim_name[i]]))
    return SF_SC

In [ ]:
SF_SC = water_content_snow()

In [ ]:
print (len(E_annual['S1']), len(Q_annual['S1']), len(SF_WC['S1']), len(SF_SC['S1']), len(Q_annual['S1']))
Total_S1 = E_annual['S1'] + Qout_annual['S1'] + SS_WC['S1'] + SF_WC['S1'] + SF_SC['S1'] 
Total_S2 = E_annual['S2'] + Qout_annual['S2'] + SS_WC['S2'] + SF_WC['S2'] + SF_SC['S2'] 
Total_S3 = E_annual['S3'] + Qout_annual['S3'] + SS_WC['S3'] + SF_WC['S3'] + SF_SC['S3'] #+ 0*Qin_annual['S3']

In [ ]:
forcing_watershed_FM = h5py.File("/Users/ajc/Core/PreProcessing/ats-intermediate-ngee/data/SNAP/CESM-RCP8.5-2006-2100_dm1985-2015-FM_2006.h5",'r')

In [ ]:
d1 = forcing_watershed_FM['precipitation rain [m s^-1]']
d2 = forcing_watershed_FM['precipitation snow [m SWE s^-1]']
rain_mm = np.reshape(d1,(-1,365)).mean(axis=1)*86400*365*1000
snow_mm = np.reshape(d2,(-1,365)).mean(axis=1)*86400*365*1000
precip_annual = snow_mm + rain_mm
r1 = np.array(d1)*86400*1000
r2 = np.array(d2)*86400*1000
precip_daily = r1 + r2

In [ ]:
plt.plot(precip_annual,'g')
plt.plot(precip_annual - Qin_annual['S3'],'m')
#plt.plot(Total_S1,'b')

#plt.plot(Total_S2,'k')
plt.plot(Total_S3,'r')